In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import efficientnet_v2_s
from sklearn.metrics import accuracy_score, f1_score
from PIL import Image
import pandas as pd
from tqdm import tqdm  # 에포크 진행 상황 확인

In [3]:
pip install wfdb


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
import wfdb
import ast
import os
from sklearn.preprocessing import MultiLabelBinarizer

def load_raw_data(df, sampling_rate, path):
    """df.index를 기준으로 데이터를 로드"""
    if sampling_rate == 100:
        data = [wfdb.rdsamp(os.path.join(path, f)) for f in df['filename_lr']]
    else:
        data = [wfdb.rdsamp(os.path.join(path, f)) for f in df['filename_hr']]
    # df.index에 있는 데이터만 로드
    data = np.array([signal for signal, meta in data])
    return data

# 데이터 경로 설정
path = "D:\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3"
sampling_rate = 100

# PTB-XL 데이터베이스 로드
df = pd.read_csv(os.path.join(path, 'ptbxl_database.csv'), index_col='ecg_id')
df.scp_codes = df.scp_codes.apply(lambda x: ast.literal_eval(x))

# 진단 정보 로드
agg_df = pd.read_csv(os.path.join(path, 'scp_statements.csv'), index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    """진단 클래스를 매핑하는 함수"""
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# 진단 클래스 매핑
df['diagnostic_superclass'] = df.scp_codes.apply(aggregate_diagnostic)

# 빈 클래스 제거
df = df[df['diagnostic_superclass'].apply(lambda x: len(x) > 0)]

# Raw data 로드
X = load_raw_data(df, sampling_rate, path)

# 크기 확인
assert len(X) == len(df), "X와 df의 크기가 일치하지 않습니다."

# 데이터셋 분리
test_fold = 10
val_fold = 9

train_filter = (df.strat_fold != test_fold) & (df.strat_fold != val_fold)
val_filter = df.strat_fold == val_fold
test_filter = df.strat_fold == test_fold

X_train = X[train_filter]
y_train = list(df[train_filter]['diagnostic_superclass'])

X_val = X[val_filter]
y_val = list(df[val_filter]['diagnostic_superclass'])

X_test = X[test_filter]
y_test = list(df[test_filter]['diagnostic_superclass'])

# 다중 라벨 이진화
mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train)
y_val_bin = mlb.transform(y_val)
y_test_bin = mlb.transform(y_test)

print(f"Train Data Shape: {X_train.shape}, Labels: {y_train_bin.shape}")
print(f"Validation Data Shape: {X_val.shape}, Labels: {y_val_bin.shape}")
print(f"Test Data Shape: {X_test.shape}, Labels: {y_test_bin.shape}")

Train Data Shape: (17084, 1000, 12), Labels: (17084, 5)
Validation Data Shape: (2146, 1000, 12), Labels: (2146, 5)
Test Data Shape: (2158, 1000, 12), Labels: (2158, 5)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import efficientnet_v2_s
import numpy as np
import matplotlib.pyplot as plt
import os

# Custom Dataset for ECG Data
class ECGDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            # ECG 데이터를 3채널로 확장
            sample = self.transform(sample)
        return sample.float(), torch.tensor(label, dtype=torch.float32)

transform = transforms.Compose([
    transforms.ToTensor(),  # Numpy 배열 -> Tensor
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.Lambda(lambda x: x.expand(3, -1, -1)),  # 1채널 데이터를 3채널로 확장
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 정규화
])


# 데이터셋 정의
train_dataset = ECGDataset(X_train, y_train_bin, transform=transform)
val_dataset = ECGDataset(X_val, y_val_bin, transform=transform)
test_dataset = ECGDataset(X_test, y_test_bin, transform=transform)

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 클래스 확인
class_names = mlb.classes_
print(f"Classes: {class_names}")

Classes: ['CD' 'HYP' 'MI' 'NORM' 'STTC']


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


## Efficientnetv2_s

In [ ]:
import torch
import torch.nn as nn
from torch import Tensor
from typing import Callable, Optional, List
from functools import partial

# Conv2dNormActivation: Convolution + Normalization + Activation
class Conv2dNormActivation(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, groups=1, norm_layer=None, activation_layer=None):
        super(Conv2dNormActivation, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, groups=groups, padding=kernel_size // 2) # 2D Convolution
        self.norm = norm_layer(out_channels) if norm_layer else nn.Identity() # 정규화 레이어
        self.activation = activation_layer() if activation_layer else nn.Identity() # 활성화 함수

    def forward(self, x: Tensor) -> Tensor:
        return self.activation(self.norm(self.conv(x))) # 순차적으로 실행


# Squeeze and Excitation Block
class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, squeeze_channels, activation=nn.SiLU):
        super(SqueezeExcitation, self).__init__()
        self.fc1 = nn.Conv2d(in_channels, squeeze_channels, 1) # 1x1 Convolution으로 채널 수 축소
        self.activation = activation() # 활성화 함수
        self.fc2 = nn.Conv2d(squeeze_channels, in_channels, 1) # 1x1 Convolution으로 채널 수 복원
        self.sigmoid = nn.Sigmoid() # Sigmoid로 중요도 계산

    def forward(self, x: Tensor) -> Tensor:
        return x * self.sigmoid(self.fc2(self.activation(self.fc1(x)))) # 중요도를 입력 특징에 반영


# Stochastic Depth
class StochasticDepth(nn.Module):
    def __init__(self, drop_prob: float, mode: str):
        super(StochasticDepth, self).__init__()
        self.drop_prob = drop_prob
        self.mode = mode

    def forward(self, x: Tensor) -> Tensor:
        if self.drop_prob > 0:
            if self.training:
                keep_prob = 1 - self.drop_prob
                mask = torch.bernoulli(torch.full((x.size(0),), keep_prob)).to(x.device).view(-1, 1, 1, 1)
                return x * mask / keep_prob # 드롭아웃 적용
            return x
        return x

# _MBConvConfig: Mobile Bottleneck Block의 설정 정보를 저장하는 클래스
class _MBConvConfig:
    def __init__(self, expand_ratio: float, kernel: int, stride: int, input_channels: int, out_channels: int, num_layers: int, block: Callable[..., nn.Module]):
        self.expand_ratio = expand_ratio
        self.kernel = kernel # 커널 크기
        self.stride = stride # 스트라이드 크기
        self.input_channels = input_channels # 입력 채널 수
        self.out_channels = out_channels # 출력 채널 수
        self.num_layers = num_layers # 레이어 반복 횟수
        self.block = block # 블록 클래스

    # 채널 수를 조정하여 8로 나누어떨어지게 만드는 유틸리티 함수
    @staticmethod
    def adjust_channels(channels: int, width_mult: float, min_value: Optional[int] = None) -> int:
        return _make_divisible(channels * width_mult, 8, min_value)


class MBConvConfig(_MBConvConfig):
    def __init__(
        self,
        expand_ratio: float,
        kernel: int,
        stride: int,
        input_channels: int,
        out_channels: int,
        num_layers: int,
        width_mult: float = 1.0,
        depth_mult: float = 1.0,
        block: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        input_channels = self.adjust_channels(input_channels, width_mult)
        out_channels = self.adjust_channels(out_channels, width_mult)
        num_layers = self.adjust_depth(num_layers, depth_mult)
        if block is None:
            block = MBConv
        super().__init__(expand_ratio, kernel, stride, input_channels, out_channels, num_layers, block)

    @staticmethod
    def adjust_depth(num_layers: int, depth_mult: float):
        return int(math.ceil(num_layers * depth_mult))


class FusedMBConvConfig(_MBConvConfig):
    def __init__(
        self,
        expand_ratio: float,
        kernel: int,
        stride: int,
        input_channels: int,
        out_channels: int,
        num_layers: int,
        block: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        if block is None:
            block = FusedMBConv
        super().__init__(expand_ratio, kernel, stride, input_channels, out_channels, num_layers, block)



# MBConv 블록 구현 (# MBConv 블록 구현 (Mobile Inverted Bottleneck Convolution 블록))
class MBConv(nn.Module):
    def __init__(self, cnf: _MBConvConfig, stochastic_depth_prob: float, norm_layer: Callable[..., nn.Module], se_layer: Callable[..., nn.Module] = SqueezeExcitation) -> None:
        super(MBConv, self).__init__()

        if not (1 <= cnf.stride <= 2):
            raise ValueError("illegal stride value") # 스트라이드 값은 1 또는 2만 허용

        self.use_res_connect = cnf.stride == 1 and cnf.input_channels == cnf.out_channels

        layers: List[nn.Module] = []
        activation_layer = nn.SiLU # 기본 활성화 함수

        # Expand: 1x1 conv, 채널 확장
        expanded_channels = cnf.adjust_channels(cnf.input_channels, cnf.expand_ratio)
        if expanded_channels != cnf.input_channels:
            layers.append(
                Conv2dNormActivation(cnf.input_channels, expanded_channels, kernel_size=1, norm_layer=norm_layer, activation_layer=activation_layer)
            )

        # Depthwise Convolution: 깊이별 연산
        layers.append(
            Conv2dNormActivation(expanded_channels, expanded_channels, kernel_size=cnf.kernel, stride=cnf.stride, groups=expanded_channels, norm_layer=norm_layer, activation_layer=activation_layer)
        )

        # Squeeze and Excitation: 채널 중요도 조정
        squeeze_channels = max(1, cnf.input_channels // 4) # 채널 축소 비율 설정
        layers.append(se_layer(expanded_channels, squeeze_channels, activation=partial(nn.SiLU, inplace=True)))

        # Project: 1x1 conv, 채널 축소
        layers.append(
            Conv2dNormActivation(expanded_channels, cnf.out_channels, kernel_size=1, norm_layer=norm_layer, activation_layer=None)
        )

        self.block = nn.Sequential(*layers) # 구성한 레이어들을 순차적으로 묶음
        self.stochastic_depth = StochasticDepth(stochastic_depth_prob, "row") # Stochastic Depth 적용
        self.out_channels = cnf.out_channels # 출력 채널 수

    def forward(self, input: Tensor) -> Tensor:
        result = self.block(input) # 블록의 연산 수행
        if self.use_res_connect: # 잔차 연결 적용 여부 확인
            result = self.stochastic_depth(result) # Stochastic Depth 적용
            result += input
        return result


# FusedMBConv (EfficientNetV2에서는 FusedMBConv를 사용)
class FusedMBConv(nn.Module):
    def __init__(self, cnf: _MBConvConfig, stochastic_depth_prob: float, norm_layer: Callable[..., nn.Module]) -> None:
        super(FusedMBConv, self).__init__()
        # stride 값이 1 또는 2가 아닌 경우 예외 발생
        if not (1 <= cnf.stride <= 2):
            raise ValueError("illegal stride value")
        
        # stride가 1이고 입력 채널과 출력 채널이 같으면 skip connection 사용
        self.use_res_connect = cnf.stride == 1 and cnf.input_channels == cnf.out_channels

        layers: List[nn.Module] = []
        activation_layer = nn.SiLU # 활성화 함수로 SiLU(Swish) 사용
        # 확장된 채널 수 계산 (expand ratio를 곱한 값)
        expanded_channels = cnf.adjust_channels(cnf.input_channels, cnf.expand_ratio)
         # 입력 채널과 확장된 채널이 다를 경우 확장 단계 추가
        if expanded_channels != cnf.input_channels:
             # 확장 단계: 확장된 채널 수를 사용한 컨볼루션 레이어 추가
            layers.append(
                Conv2dNormActivation(cnf.input_channels, expanded_channels, kernel_size=cnf.kernel, stride=cnf.stride, norm_layer=norm_layer, activation_layer=activation_layer)
            )

            # Project: 1x1 Pointwise Conv로 채널 수를 출력 채널로 변경
            layers.append(
                Conv2dNormActivation(expanded_channels, cnf.out_channels, kernel_size=1, norm_layer=norm_layer, activation_layer=None)
            )
        else:
            # 확장이 필요 없는 경우 단일 컨볼루션 레이어로 처리
            layers.append(
                Conv2dNormActivation(cnf.input_channels, cnf.out_channels, kernel_size=cnf.kernel, stride=cnf.stride, norm_layer=norm_layer, activation_layer=activation_layer)
            )
        # 생성된 레이어 리스트를 nn.Sequential로 묶어 블록 생성
        self.block = nn.Sequential(*layers)

         # Stochastic Depth 적용
        self.stochastic_depth = StochasticDepth(stochastic_depth_prob, "row")
        
        # 출력 채널 저장
        self.out_channels = cnf.out_channels

    def forward(self, input: Tensor) -> Tensor:
        result = self.block(input)
        if self.use_res_connect:
            # Stochastic Depth로 일부 입력 무작위 드롭
            result = self.stochastic_depth(result)
            result += input
        return result

# 주어진 값(value)을 특정 값(divisor)로 나누어 떨어지도록 조정하는 함수
def _make_divisible(value: int, divisor: int, min_value: Optional[int] = None) -> int:
    if min_value is None:
        min_value = divisor # 최소값이 지정되지 않았으면 divisor 사용
    new_value = max(min_value, int(value + divisor / 2) // divisor * divisor)
     # 조정된 값이 원래 값의 90% 미만이면 divisor를 더해 조정
    if new_value < 0.9 * value:
        new_value += divisor
    return new_value


# FusedMBConvConfig 객체로 inverted_residual_setting을 구성
inverted_residual_setting = [
    FusedMBConvConfig(1, 3, 2, 32, 16, 2),
    FusedMBConvConfig(2, 3, 2, 16, 32, 4),
    FusedMBConvConfig(3, 3, 2, 32, 64, 4),
    FusedMBConvConfig(4, 3, 1, 64, 128, 6),
    FusedMBConvConfig(5, 3, 2, 128, 256, 15),
]

In [ ]:
from typing import Any, Callable, List, Optional, Sequence, Union, Tuple
import math
import copy
import torch
import torch.nn as nn
from functools import partial

# WeightsEnum에 필요한 경우 사용할 수 있는 자리 표시자 클래스
class WeightsEnum:
    pass

# 학습된 가중치를 저장하는 Weights 클래스 정의
class Weights:
    def __init__(self, url: str, transforms: Callable, meta: dict):
        self.url = url
        self.transforms = transforms
        self.meta = meta

# 이미지 분류를 위한 데이터 변환 클래스
class ImageClassification:
    def __init__(self, crop_size: int, resize_size: int, interpolation: str):
        self.crop_size = crop_size  # 자를 이미지 크기
        self.resize_size = resize_size # 리사이즈할 이미지 크기
        self.interpolation = interpolation # 이미지 보간(interpolation) 방식

# EfficientNet V2 모델의 가중치 Enum 정의
class EfficientNet_V2_S_Weights(WeightsEnum):
    IMAGENET1K_V1 = Weights(
        url="https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth",
        transforms=partial(
            ImageClassification,
            crop_size=384,
            resize_size=384,
            interpolation="BILINEAR",  # You can also import InterpolationMode.BILINEAR if you prefer
        ),
        meta={
            "num_params": 21458488,
            "_metrics": {
                "ImageNet-1K": {
                    "acc@1": 84.228,
                    "acc@5": 96.878,
                }
            },
            "_ops": 8.366,
            "_file_size": 82.704,
            "_docs": """
                These weights improve upon the results of the original paper by using a modified version of TorchVision's
                `new training recipe
                <https://pytorch.org/blog/how-to-train-state-of-the-art-models-using-torchvision-latest-primitives/>`_.
            """,
        },
    )
    DEFAULT = IMAGENET1K_V1  # 기본 가중치 설정

# EfficientNet V2 모델 클래스 정의
class EfficientNetV2(nn.Module):
    def __init__(
        self,
        inverted_residual_setting: Sequence[Union[MBConvConfig, FusedMBConvConfig]],
        dropout: float,
        stochastic_depth_prob: float = 0.2, # 확률적으로 레이어를 드롭
        num_classes: int = 1000, # 출력 클래스 수
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        last_channel: Optional[int] = None, # 마지막 출력 채널 수
    ) -> None:
        super().__init__()

        # 입력 검증
        if not inverted_residual_setting:
            raise ValueError("The inverted_residual_setting should not be empty")
        elif not isinstance(inverted_residual_setting, Sequence) or not all([isinstance(s, _MBConvConfig) for s in inverted_residual_setting]):
            raise TypeError("The inverted_residual_setting should be List[MBConvConfig]")
        
        # 기본 정규화 레이어를 BatchNorm2d로 설정
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        layers: List[nn.Module] = [] # 모델 레이어를 저장할 리스트
        
        # 초기 Conv 레이어 추가
        firstconv_output_channels = inverted_residual_setting[0].input_channels
        layers.append(
            Conv2dNormActivation(
                3, firstconv_output_channels, kernel_size=3, stride=2, norm_layer=norm_layer, activation_layer=nn.SiLU
            )
        )
        # MBConv 블록 추가
        total_stage_blocks = sum(cnf.num_layers for cnf in inverted_residual_setting)
        stage_block_id = 0
        for cnf in inverted_residual_setting:
            stage: List[nn.Module] = []
            for _ in range(cnf.num_layers):
                block_cnf = copy.copy(cnf)
                if stage:
                    block_cnf.input_channels = block_cnf.out_channels
                    block_cnf.stride = 1
                sd_prob = stochastic_depth_prob * float(stage_block_id) / total_stage_blocks
                stage.append(block_cnf.block(block_cnf, sd_prob, norm_layer))
                stage_block_id += 1
            layers.append(nn.Sequential(*stage))
        # 마지막 Conv 레이어 추가
        lastconv_input_channels = inverted_residual_setting[-1].out_channels
        lastconv_output_channels = last_channel if last_channel is not None else 4 * lastconv_input_channels
        layers.append(
            Conv2dNormActivation(
                lastconv_input_channels,
                lastconv_output_channels,
                kernel_size=1,
                norm_layer=norm_layer,
                activation_layer=nn.SiLU,
            )
        )

        # 특징 추출 파트 설정
        self.features = nn.Sequential(*layers)
        self.avgpool = nn.AdaptiveAvgPool2d(1) # Adaptive Average Pooling
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout, inplace=True),
            nn.Linear(lastconv_output_channels, 5),  # num_classes를 5로 설정
            )

        # 가중치 초기화
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out")
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                init_range = 1.0 / math.sqrt(m.out_features)
                nn.init.uniform_(m.weight, -init_range, init_range)
                nn.init.zeros_(m.bias)
    # 모델 순전파 구현 (내부용)
    def _forward_impl(self, x: Tensor) -> Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    # forward 함수
    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

# EfficientNet 생성 함수
def _efficientnet(
    inverted_residual_setting: Sequence[Union[MBConvConfig, FusedMBConvConfig]],
    dropout: float,
    last_channel: Optional[int],
    weights: Optional[WeightsEnum], # 사전 학습된 가중치
    progress: bool,
    **kwargs: Any,
) -> EfficientNetV2:
    if weights is not None:
        _ovewrite_named_param(kwargs, "num_classes", len(weights.meta["categories"]))

    model = EfficientNetV2(inverted_residual_setting, dropout, last_channel=last_channel, **kwargs)

    if weights is not None:
        model.load_state_dict(weights.get_state_dict(progress=progress, check_hash=True))

    return model


### 1) Early stopping 포함 8 epoch까지 학습한 결과

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from torchvision import models
from functools import partial

# EfficientNetV2-S 모델 설정 및 가중치 불러오기
from torchvision.models import EfficientNet_V2_S_Weights

def _efficientnet_conf(
    arch: str,
    **kwargs: Any,
) -> Tuple[Sequence[Union[MBConvConfig, FusedMBConvConfig]], Optional[int]]:
    inverted_residual_setting: Sequence[Union[MBConvConfig, FusedMBConvConfig]]
    
    # 'efficientnet_v2_s' 아키텍처만 사용
    if arch == "efficientnet_v2_s":
        inverted_residual_setting = [
            FusedMBConvConfig(1, 3, 1, 24, 24, 2),
            FusedMBConvConfig(4, 3, 2, 24, 48, 4),
            FusedMBConvConfig(4, 3, 2, 48, 64, 4),
            MBConvConfig(4, 3, 2, 64, 128, 6),
            MBConvConfig(6, 3, 1, 128, 160, 9),
            MBConvConfig(6, 3, 2, 160, 256, 15),
        ]
        last_channel = 1280
    else:
        raise ValueError(f"Unsupported architecture: {arch}")

    return inverted_residual_setting, last_channel


def efficientnet_v2_s(
    *, weights: Optional[EfficientNet_V2_S_Weights] = None, progress: bool = True, **kwargs: Any
) -> EfficientNetV2:
    """
    Constructs an EfficientNetV2-S architecture and loads pre-trained weights.
    """
    weights = EfficientNet_V2_S_Weights.verify(weights)

    inverted_residual_setting, last_channel = _efficientnet_conf("efficientnet_v2_s")
    model = _efficientnet(
        inverted_residual_setting,
        kwargs.pop("dropout", 0.2),
        last_channel,
        weights,
        progress,
        norm_layer=partial(nn.BatchNorm2d, eps=1e-03),
        **kwargs,
    )
    return model

# 모델 불러오기
# EfficientNetV2 모델에 설정을 전달
model = EfficientNetV2(
    inverted_residual_setting=inverted_residual_setting,
    dropout=0.2,
    num_classes=5
)

# Loss Function, Optimizer 설정
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# EarlyStopping 클래스 정의
class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.best_score = None
        self.epochs_no_improve = 0
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, score, model):
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(score, model)
        elif score < self.best_score + self.delta:
            self.epochs_no_improve += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.epochs_no_improve} out of {self.patience}')
            if self.epochs_no_improve >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(score, model)
            self.epochs_no_improve = 0

        return self.early_stop

    def save_checkpoint(self, score, model):
        if self.verbose:
            print(f'Validation score improved ({self.best_score:.6f} --> {score:.6f}). Saving model...')
        self.best_model_wts = model.state_dict()

# 모델 학습과 평가 함수 정의
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    predictions, true_labels = [], []

    for images, labels in tqdm(dataloader, desc="Training", leave=False, ncols=100):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        predictions.extend(torch.sigmoid(outputs).cpu().detach().numpy() > 0.5)
        true_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(dataloader)
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average="macro")

    return avg_loss, accuracy, f1

def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    predictions, true_labels = [], []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating", leave=False, ncols=100):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            predictions.extend(torch.sigmoid(outputs).cpu().numpy() > 0.5)
            true_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(dataloader)
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average="macro")

    return avg_loss, accuracy, f1

# 모델 학습 실행
num_epochs = 20
patience = 3  # 조기 중지 시 허용되는 에폭 수
best_loss = float('inf')  # 초기값을 무한대로 설정

# EarlyStopping 객체 초기화
early_stopping = EarlyStopping(patience=patience, verbose=True)

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Train
    train_loss, train_acc, train_f1 = train_model(model, train_loader, criterion, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}, F1 Score: {train_f1:.4f}")

    # Validation
    val_loss, val_acc, val_f1 = evaluate_model(model, val_loader, criterion, device)
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}, F1 Score: {val_f1:.4f}")

    # EarlyStopping 체크
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}, F1 Score: {val_f1:.4f}')
    
    # 모델 성능 개선 여부 확인
    if val_loss < best_loss:
        best_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    # Early stopping 기준: F1 Score를 기준으로 EarlyStopping 체크
    if early_stopping(val_f1, model):
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break

# 최상의 모델 불러오기
model.load_state_dict(early_stopping.best_model_wts)

# Test 모델 평가
test_loss, test_accuracy, test_f1 = evaluate_model(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}, F1 Score: {test_f1:.4f}")


Epoch 1/20


Train Loss: 0.5457, Accuracy: 0.2956, F1 Score: 0.3008


Validation Loss: 0.7002, Accuracy: 0.4338, F1 Score: 0.2181
Epoch [1/20], Loss: 0.7002, Accuracy: 0.4338, F1 Score: 0.2181
Validation score improved (0.218105 --> 0.218105). Saving model...
Epoch 2/20


Train Loss: 0.4280, Accuracy: 0.3928, F1 Score: 0.4567


Validation Loss: 0.4044, Accuracy: 0.4464, F1 Score: 0.4235
Epoch [2/20], Loss: 0.4044, Accuracy: 0.4464, F1 Score: 0.4235
Validation score improved (0.423494 --> 0.423494). Saving model...
Epoch 3/20


...

Train Loss: 0.3317, Accuracy: 0.5387, F1 Score: 0.6516


Validation Loss: 0.3485, Accuracy: 0.5242, F1 Score: 0.5440
Epoch [7/20], Loss: 0.3485, Accuracy: 0.5242, F1 Score: 0.5440
EarlyStopping counter: 2 out of 3
Epoch 8/20


Train Loss: 0.3257, Accuracy: 0.5483, F1 Score: 0.6602


Validation Loss: 0.3435, Accuracy: 0.4921, F1 Score: 0.5892
Epoch [8/20], Loss: 0.3435, Accuracy: 0.4921, F1 Score: 0.5892
EarlyStopping counter: 3 out of 3
Early stopping triggered after 8 epochs.


Test Loss: 0.3545, Accuracy: 0.4884, F1 Score: 0.5938


### 2) Early stopping 없이 10 epoch 학습한 결과

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from torchvision import models
from functools import partial

# EfficientNetV2-S 모델 설정 및 가중치 불러오기
from torchvision.models import EfficientNet_V2_S_Weights

def _efficientnet_conf(
    arch: str,
    **kwargs: Any,
) -> Tuple[Sequence[Union[MBConvConfig, FusedMBConvConfig]], Optional[int]]:
    inverted_residual_setting: Sequence[Union[MBConvConfig, FusedMBConvConfig]]
    
    # 'efficientnet_v2_s' 아키텍처만 사용
    if arch == "efficientnet_v2_s":
        inverted_residual_setting = [
            FusedMBConvConfig(1, 3, 1, 24, 24, 2),
            FusedMBConvConfig(4, 3, 2, 24, 48, 4),
            FusedMBConvConfig(4, 3, 2, 48, 64, 4),
            MBConvConfig(4, 3, 2, 64, 128, 6),
            MBConvConfig(6, 3, 1, 128, 160, 9),
            MBConvConfig(6, 3, 2, 160, 256, 15),
        ]
        last_channel = 1280
    else:
        raise ValueError(f"Unsupported architecture: {arch}")

    return inverted_residual_setting, last_channel


def efficientnet_v2_s(
    *, weights: Optional[EfficientNet_V2_S_Weights] = None, progress: bool = True, **kwargs: Any
) -> EfficientNetV2:
    """
    Constructs an EfficientNetV2-S architecture and loads pre-trained weights.
    """
    weights = EfficientNet_V2_S_Weights.verify(weights)

    inverted_residual_setting, last_channel = _efficientnet_conf("efficientnet_v2_s")
    model = _efficientnet(
        inverted_residual_setting,
        kwargs.pop("dropout", 0.2),
        last_channel,
        weights,
        progress,
        norm_layer=partial(nn.BatchNorm2d, eps=1e-03),
        **kwargs,
    )
    return model

# 모델 불러오기
# EfficientNetV2 모델에 설정을 전달
model = EfficientNetV2(
    inverted_residual_setting=inverted_residual_setting,
    dropout=0.2,
    num_classes=5
)

# Loss Function, Optimizer 설정
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)


# 모델 학습과 평가 함수 정의
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    predictions, true_labels = [], []

    for images, labels in tqdm(dataloader, desc="Training", leave=False, ncols=100):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        predictions.extend(torch.sigmoid(outputs).cpu().detach().numpy() > 0.5)
        true_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(dataloader)
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average="macro")

    return avg_loss, accuracy, f1

def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    predictions, true_labels = [], []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating", leave=False, ncols=100):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            predictions.extend(torch.sigmoid(outputs).cpu().numpy() > 0.5)
            true_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(dataloader)
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average="macro")

    return avg_loss, accuracy, f1

# 모델 학습 실행
num_epochs = 10
best_val_loss = float('inf')  # 초기값을 무한대로 설정
best_model_weights = None

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Train
    train_epoch_loss, train_epoch_acc, train_epoch_f1 = train_model(
        model, train_loader, criterion, optimizer, device
    )
    print(
        f"Train Loss: {train_epoch_loss:.4f}, "
        f"Accuracy: {train_epoch_acc:.4f}, "
        f"F1 Score: {train_epoch_f1:.4f}"
    )

    # Validation
    val_epoch_loss, val_epoch_acc, val_epoch_f1 = evaluate_model(
        model, val_loader, criterion, device
    )
    print(
        f"Validation Loss: {val_epoch_loss:.4f}, "
        f"Accuracy: {val_epoch_acc:.4f}, "
        f"F1 Score: {val_epoch_f1:.4f}"
    )

    # 최적 성능 모델의 가중치 저장
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        best_model_weights = model.state_dict()

# 최상의 모델 불러오기
model.load_state_dict(best_model_weights)

# Test 모델 평가
test_loss, test_acc, test_f1 = evaluate_model(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, F1 Score: {test_f1:.4f}")


Epoch 1/10


Train Loss: 0.5584, Accuracy: 0.2779, F1 Score: 0.2806


Validation Loss: 0.5302, Accuracy: 0.1650, F1 Score: 0.3306
Epoch 2/10


Train Loss: 0.4298, Accuracy: 0.3857, F1 Score: 0.4485


Validation Loss: 0.3934, Accuracy: 0.4832, F1 Score: 0.4950
Epoch 3/10


...

Train Loss: 0.3138, Accuracy: 0.5662, F1 Score: 0.6760


Validation Loss: 0.3319, Accuracy: 0.5256, F1 Score: 0.6951
Epoch 10/10


Train Loss: 0.3091, Accuracy: 0.5745, F1 Score: 0.6850


Validation Loss: 0.3285, Accuracy: 0.5788, F1 Score: 0.6514


Test Loss: 0.3428, Accuracy: 0.5765, F1 Score: 0.6384
